# 캐릭터 조종2 및 액션

전체 코드

In [2]:
import numpy as np
from digitalio import DigitalInOut, Direction
from adafruit_rgb_display import st7789
import board
from PIL import Image, ImageDraw, ImageFont, ImageOps
import time
import random
import cv2 as cv
import numpy as np
from colorsys import hsv_to_rgb

class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins:
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        self.width = self.disp.width
        self.height = self.disp.height


level1Fish = [
    "fish/f_1_1.png", "fish/f_1_2.png", "fish/f_1_3.png", "fish/f_1_4.png", "fish/f_1_5.png"]
level2Fish = [
    "fish/f_2_1.png", "fish/f_2_2.png", "fish/f_2_3.png"]
level3Fish = [
    "fish/f_3_1.png", "fish/f_3_2.png", "fish/f_3_3.png"]

class Fish:
    def __init__(self, level, width, height):
        self.level = level
        self.width = width
        self.height = height
        self.generate_random_fish()

    def generate_random_fish(self):
        fish_levels = [level1Fish, level2Fish, level3Fish]
        fish_images = random.choice(fish_levels[:self.level])
        fish_sizes = [(25, 25), (40, 40), (55, 55)]  # 레벨에 따른 물고기 크기
        base_image = Image.open(random.choice(fish_images)).resize(fish_sizes[self.level - 1])
        base_image = ImageOps.flip(base_image) 
        
        self.appearances = {
            'left': ImageOps.mirror(base_image),
            'right': base_image
        }
        self.appearance = self.appearances[random.choice(['left', 'right'])]
        self.position = np.array([random.randint(0, self.width - fish_sizes[self.level - 1][0]), random.randint(0, self.height - fish_sizes[self.level - 1][1]),
                                  random.randint(0, self.width - fish_sizes[self.level - 1][0]) + fish_sizes[self.level - 1][0], random.randint(0, self.height - fish_sizes[self.level - 1][1]) + fish_sizes[self.level - 1][1]])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#FFFFFF"

    def move(self):
        if self.appearance == self.appearances['left']:
            self.position[0] -= 3
            self.position[2] -= 3
            if self.position[0] <= 0:
                self.appearance = self.appearances['right']
        else:
            self.position[0] += 3
            self.position[2] += 3
            if self.position[2] >= self.width:
                self.appearance = self.appearances['left']
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])


class Nemo:
    def __init__(self, width, height):
        # 기본 이미지 불러오기
        base_image = Image.open('fish/f_nemo.png').resize((40, 40))

        # 각 방향에 대한 이미지 미리 로드 및 회전
        self.appearances = {  
            #업다운은 빼도 될 듯?
            'up': base_image.rotate(270),
            'down': base_image.rotate(90),
            'left': ImageOps.mirror(base_image.rotate(180)),
            'right': base_image.rotate(180)  # 오른쪽을 보는 이미지는 왼쪽 이미지를 좌우 반전
        }
        self.appearance = self.appearances['left']  # 기본적으로 아래를 보도록 설정

        self.state = None
        self.position = np.array([width/2 - 20, height/2 - 20, width/2 + 20, height/2 + 20])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#FFFFFF"

    def move(self, command=None):
        if command['move'] == False:
            self.state = None
            self.outline = "#FFFFFF"  # 검정색상 코드!
        else:
            self.state = 'move'
            self.outline = "#FF0000"  # 빨강색상 코드!

            if command['up_pressed']:
                self.position[1] -= 5
                self.position[3] -= 5
                self.appearance = self.appearances['up']  # 위쪽으로 움직일 때 이미지 변경

            if command['down_pressed']:
                self.position[1] += 5
                self.position[3] += 5
                self.appearance = self.appearances['down']  # 아래쪽으로 움직일 때 이미지 변경

            if command['left_pressed']:
                self.position[0] -= 5
                self.position[2] -= 5
                self.appearance = self.appearances['left']  # 왼쪽으로 움직일 때 이미지 변경

            if command['right_pressed']:    
                self.position[0] += 5
                self.position[2] += 5
                self.appearance = self.appearances['right']  # 오른쪽으로 움직일 때 이미지 변경
                
        #center update
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2]) 


def main():
    joystick = Joystick()
    my_image = Image.new("RGB", (joystick.width, joystick.height))
    bg_image = Image.open('bg/bg1.png').resize((joystick.width, joystick.height))  # 배경 이미지 파일 불러오기 및 크기 조정
    bg_image = bg_image.rotate(180)  # 배경 이미지 180도 회전

    my_draw = ImageDraw.Draw(my_image)
    my_Nemo = Nemo(joystick.width, joystick.height)

    fish_list = []
    last_fish_time = time.time()

    while True:
        command = {'move': False, 'up_pressed': False , 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
        
        if not joystick.button_U.value:  # up pressed
            command['up_pressed'] = True
            command['move'] = True

        if not joystick.button_D.value:  # down pressed
            command['down_pressed'] = True
            command['move'] = True

        if not joystick.button_L.value:  # left pressed
            command['left_pressed'] = True
            command['move'] = True

        if not joystick.button_R.value:  # right pressed
            command['right_pressed'] = True
            command['move'] = True

        my_Nemo.move(command)

        current_time = time.time()
        if current_time - last_fish_time > 5:  # 3초마다 물고기 생성
            fish_list.append(Fish(1, joystick.width, joystick.height))
            last_fish_time = current_time

        for fish in fish_list:
            fish.move()

        my_image.paste(bg_image)  # 배경 이미지 그리기
        my_image.paste(my_Nemo.appearance, (int(my_Nemo.position[0]), int(my_Nemo.position[1])))  # 캐릭터 이미지 그리기

        for fish in fish_list:
            my_image.paste(fish.appearance, (int(fish.position[0]), int(fish.position[1])))  # 물고기 이미지 그리기

        joystick.disp.image(my_image)

if __name__ == '__main__':
    main()


KeyboardInterrupt: 